## P&O ISSP: Brain-computer interface voor sturing van een directionele akoestische zoom

In this notebook, we will start building a basic deep learning implementation for classifying which two of the Stimuli was attended to, when given EEG and both Stimuli as input. 


One of the ways to process the EEG data is to find specific patterns in the signal. Based on the presence or absence of these patterns we will decide where is the auditory attention. But handcrafting these pattern might be difficult, so we will use convolutional neural network to learn filters which can detect those patterns.

The implementation will be in mulitple phases. First, we will get more familiar with keras and the deep learning framework by mimicking the linear regression-based network, but then in a non-linear context. Once we have implemented this, we can start playing with the deep learning architectures and add some blocks, see what different training schemes do, etc... 

The following paper has a very similar approach to the one we will take. Note that there is a difference with the data. In the paper, there is always only 1 speaker that is talking while the EEG is recorded, whereas we have 2 competing speakers. However, the implementation of the networks will be the same. Instead of taking the mismatch in the future of the same envelope, we already have an attended and unattended envelope here. 

Accou, Bernd, et al. "Modeling the relationship between acoustic stimulus and EEG with a dilated convolutional neural network." 2020 28th European Signal Processing Conference (EUSIPCO). IEEE, 2021.

If we look at the paper, they compare three different methods. We have alread implemented method a) Linear decoder baseline. 
We will now first implement method b) Convolutional baseline method and then expand from this towards c) dilation model or d) your own extentions on this model 


**Note**: If keras is  not already installed, execute: !pip install keras

In [ ]:
# Load required libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Activation
from keras import regularizers

![fig_1](dilation-figures.png)

* The EEG data preprocessing has been explained in another tutorial.
* we have already implemeted method a) linear decoder baseline 
* we will now start with method b) convolutional baseline method 
* once this is implemented, we can expand this model towards model c) dilation model and possibly add extentions 

**Convolutional baseline network**
* The first step in the model is a convolutional layer, indicated in red. A (64 x 16) spatio-temporal filter is shifted over the input matrix, containing the EEG.
* A rectifying linear unit (ReLu) activation function is used after the convolution step. the kernel size of 16 is chosen because, as is the case in the linear model, we want to look to future EEG to predict the current envelope. the EEG is sampled at fs=64Hz, giving us a temporal resolution of 16/64 = 250ms.
* The output of the convolutional block is a (time-window, 1) signal. 
* In the next step, we calculate the cosine similarity between this signal and both of the envelopes. We will calculate this cosine similarity by applying a *dot product* between the signal and both envelopes. 
* As a last step, we then have to choose which one of the two attended envelopes is the one we want to choose. We do this by applying a single neuron ( **dense layer** in keras, with a sigmoid activation function. 

**dilation model**
* the idea here is the same. We still give EEG and envelopes to the model, there are just more processing steps in between before we have to make a decision. 
* we first apply a one-dimensional convolution to the EEG, with 8 output filters. We can interpret this as kind of a non-linear dimensionality reduction, as the resulting EEG has shape (time-window, 8) instead of the original (time-window, 64) 
* next, there are some dilated convolutional blocks. these blocks will perform convolutions, with a certain dilation factor ( see picture). This enlarges the receptive field of the convolution, while still keeping the number of parameters small. Eg. we can look over a longer time period. These convolutions are applied to both EEG and envelopes. 
* after that, we once again compute the dot product and subsequently put the result of this in a sigmoid neuron to reach an end decision. 





In [ ]:

eeg = tf.keras.layers.Input(shape=[time_window, 64])
env1 = tf.keras.layers.Input(shape=[time_window, 1])
env2 = tf.keras.layers.Input(shape=[time_window, 1])

#add model layers
## ---- add your code ----here

# Classification
out1 = tf.keras.layers.Dense(1, activation="sigmoid")(
    tf.keras.layers.Flatten()(tf.keras.layers.Concatenate()([cos1, cos2])))

# 1 output per batch
out = tf.keras.layers.Reshape([1], name=output_name)(out1)
model = tf.keras.Model(inputs=[eeg, env1, env2], outputs=[out])



In [ ]:
# To check the model summary:
model.summary()

Before we start training the model, we need to make sure that the data is equally balanced. We have attended and unattended envelopes that we give to the model. If we always put the attended envelope at stream 1 and the unattended at stream 2, the model will quickly figure out that it should just always output stream 1 and hence not learn anything. 

The solution to this is to present each segment of EEG twice, where we swap the envelopes, ( and thus, the labels), from place 

In [ ]:

def batch_equalizer(eeg, env_1, env_2, labels):
    # present each of the eeg segments twice, where the envelopes, and thus the labels 
    # are swapped around. EEG presented in small segments [bs, window_length, 64]
    return (np.concatenate([eeg,eeg], axis=0), np.concatenate([env_1, env_2], axis=0),np.concatenate([ env_2, env_1], axis=0)), np.concatenate([labels, (labels+1)%2], axis=0)


* Now we prepare our data to train the model.

In [ ]:
import h5py
import numpy as np

# To load the mat file in v7.3 format. For all previous formats use scipy.io loadmat (https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html)
def load_large_mat(filepath):
    arrays = {}
    f = h5py.File(filepath)
    for k, v in f.items():
        arrays[k] = np.array(v)
    f.close()
    return arrays

In [ ]:
# Preprocessing
def fn_all(fnarrays1,fnarrays2):
    #------add your prerprocessing steps here
    fnxtr_all = fnarrays1
    fny_tr_all = fnarrays2
    x = np.expand_dims(fnxtr_all,-1)
    
    return x,y